In [0]:
FilePath3= "/FileStore/tables/clinicaltrial_2021_csv.gz"
dbutils.fs.cp(FilePath3, "file:/tmp/")

Out[2]: True

In [0]:
%sh
gunzip /tmp/clinicaltrial_2021_csv.gz

In [0]:
dbutils.fs.cp("file:/tmp/clinicaltrial_2021_csv", "file:/tmp/clinicaltrial_2021.csv")
dbutils.fs.mv("file:/tmp/clinicaltrial_2021.csv", "FileStore/tables/")

Out[4]: True

In [0]:
DF21 = spark.read.csv("dbfs:/FileStore/tables/clinicaltrial_2021.csv", sep = "|", header=True).distinct()
DF21.show()

+-----------+--------------------+--------------------+--------+----------+--------------------+----------+--------------------+--------------------+
|         Id|             Sponsor|              Status|   Start|Completion|                Type|Submission|          Conditions|       Interventions|
+-----------+--------------------+--------------------+--------+----------+--------------------+----------+--------------------+--------------------+
|NCT02757209|Consorzio Futuro ...|           Completed|Apr 2016|  Jan 2018|      Interventional|  Apr 2016|              Asthma|Fluticasone,Xhanc...|
|NCT02751762|Member Companies ...|          Recruiting|Nov 2017|  Oct 2022|       Observational|  Apr 2016|Chronic Pain,Subs...|                null|
|NCT02751957|     Duke University|           Completed|Jul 2016|  Jul 2020|      Interventional|  Apr 2016|Autistic Disorder...|                null|
|NCT02758860|University of Rom...|Active, not recru...|Jun 2016|  Sep 2020|Observational [Pa...|  Ap

In [0]:
DF21.count()

Out[6]: 387261

In [0]:
DF21Type = DF21.select("Type")
DF21Type.show()

+--------------------+
|                Type|
+--------------------+
|      Interventional|
|       Observational|
|      Interventional|
|Observational [Pa...|
|      Interventional|
|       Observational|
|Observational [Pa...|
|      Interventional|
|      Interventional|
|       Observational|
|      Interventional|
|      Interventional|
|      Interventional|
|      Interventional|
|       Observational|
|       Observational|
|      Interventional|
|      Interventional|
|      Interventional|
|       Observational|
+--------------------+
only showing top 20 rows



In [0]:
DF21TypeFreq = DF21Type.groupBy(['Type']).count()
DF21TypeFreq.sort("count", ascending=False).show()

+--------------------+------+
|                Type| count|
+--------------------+------+
|      Interventional|301472|
|       Observational| 77540|
|Observational [Pa...|  8180|
|     Expanded Access|    69|
+--------------------+------+



In [0]:
from pyspark.sql.functions import split, explode
DF21Con = DF21.select("Conditions").withColumn(("Conditions"), explode(split("Conditions", ",")))
DF21ConFreq = DF21Con.groupBy("Conditions").count()
DF21ConFreq.sort("count", ascending=False).show(5)

+-----------------+-----+
|       Conditions|count|
+-----------------+-----+
|        Carcinoma|13389|
|Diabetes Mellitus|11080|
|        Neoplasms| 9371|
| Breast Neoplasms| 8640|
|         Syndrome| 8032|
+-----------------+-----+
only showing top 5 rows

